## Setup and Configuration

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.sparse.linalg import eigs
import time
import gc  # Garbage collection for memory management
from getParam_Sonar import getParam_Sonar
from eval_f_Sonar import eval_f_Sonar
from eval_u_Sonar import eval_u_Sonar_20
from eval_g_Sonar import eval_g_Sonar
from simpleLeapFrog import LeapfrogSolver
from visualize_sonar import visualize_sonar_setup
from create_wave_animation import create_wave_animation
import scipy.sparse as sp
import os
import pickle

In [2]:
# Define the three grid configurations (optimized for 4 ppw @ 2 kHz)
grid_configs = {
    'small': {'Nx': 101, 'Nz': 51, 'Lx': 3000, 'Lz': 1500},
    'medium': {'Nx': 201, 'Nz': 101, 'Lx': 3000, 'Lz': 1500},
    'large': {'Nx': 361, 'Nz': 181, 'Lx': 3000, 'Lz': 1500}
}

# Initialize models
models = {}

# Signal properties from eval_u_Sonar
f_signal = 20  # Hz
wavelength = 1500 / f_signal
dt_nyquist = 1 / (2 * f_signal)

print("="*80)
print("Setting up Sonar Models")
print("="*80)

for name, config in grid_configs.items():
    Nx, Nz = config['Nx'], config['Nz']
    
    # Get parameters
    p, x_start, t_start, t_stop, max_dt_FE = getParam_Sonar(
        Nx, Nz, config['Lx'], config['Lz'], UseSparseMatrices=True, alpha=0.1, BC=True
    )
    
    N = Nx * Nz

    # Override t_stop for faster convergence analysis (keep memory manageable)
    t_stop = 1.5
    
    # Configure hydrophones at 1/3 and 2/3 of X domain
    hydro_1_x = Nx // 3
    hydro_2_x = 2 * Nx // 3
    
    # Source co-located with first hydrophone
    p['sonar_ix'] = hydro_1_x
    p['sonar_iz'] = Nz // 2
    source_idx = p['sonar_ix'] * Nz + p['sonar_iz']
    
    # Rebuild B matrix with new source location
    B_lil = sp.lil_matrix((2*N, 1), dtype=float)
    B_lil[source_idx, 0] = 1.0 / (p['dx'] * p['dz'])
    p['B'] = B_lil.tocsr()
    
    # Configure hydrophones
    p['hydrophones'] = {
        'z_pos': p['sonar_iz'],
        'x_indices': [hydro_1_x, hydro_2_x],
        'n_phones': 2
    }
    
    # Store model
    models[name] = {
        'p': p,
        'x_start': x_start,
        't_start': t_start,
        't_stop': t_stop,
        'max_dt_FE': max_dt_FE,
        'config': config
    }
    
    # Calculate metrics
    N_states = 2 * N
    ppw = wavelength / p['dx']
    nyquist_ok = max_dt_FE <= dt_nyquist
    separation_m = (hydro_2_x - hydro_1_x) * p['dx']
    delay_ms = separation_m / p['c'] * 1e3
    
    print(f"\n{name.upper()}: {Nx}×{Nz} = {N_states:,} states")
    print(f"  Domain: {config['Lx']:.1f}×{config['Lz']:.1f} m, dx={p['dx']*1e3:.1f} mm")
    print(f"  Resolution: {ppw:.1f} ppw, CFL={max_dt_FE*1e6:.1f} μs")
    print(f"  Nyquist: {'✓ PASS' if nyquist_ok else '✗ FAIL'} (limit={dt_nyquist*1e6:.1f} μs)")
    print(f"  Source/H1 at x={hydro_1_x*p['dx']:.1f}m, H2 at x={hydro_2_x*p['dx']:.1f}m")
    print(f"  Separation: {separation_m:.1f}m → delay ≈ {delay_ms:.2f} ms")

print("\n" + "="*80)

Setting up Sonar Models

SMALL: 101×51 = 10,302 states
  Domain: 3000.0×1500.0 m, dx=30000.0 mm
  Resolution: 2.5 ppw, CFL=7071.1 μs
  Nyquist: ✓ PASS (limit=25000.0 μs)
  Source/H1 at x=990.0m, H2 at x=2010.0m
  Separation: 1020.0m → delay ≈ 680.00 ms

MEDIUM: 201×101 = 40,602 states
  Domain: 3000.0×1500.0 m, dx=15000.0 mm
  Resolution: 5.0 ppw, CFL=3535.5 μs
  Nyquist: ✓ PASS (limit=25000.0 μs)
  Source/H1 at x=1005.0m, H2 at x=2010.0m
  Separation: 1005.0m → delay ≈ 670.00 ms

MEDIUM: 201×101 = 40,602 states
  Domain: 3000.0×1500.0 m, dx=15000.0 mm
  Resolution: 5.0 ppw, CFL=3535.5 μs
  Nyquist: ✓ PASS (limit=25000.0 μs)
  Source/H1 at x=1005.0m, H2 at x=2010.0m
  Separation: 1005.0m → delay ≈ 670.00 ms

LARGE: 361×181 = 130,682 states
  Domain: 3000.0×1500.0 m, dx=8333.3 mm
  Resolution: 9.0 ppw, CFL=1964.2 μs
  Nyquist: ✓ PASS (limit=25000.0 μs)
  Source/H1 at x=1000.0m, H2 at x=2000.0m
  Separation: 1000.0m → delay ≈ 666.67 ms


LARGE: 361×181 = 130,682 states
  Domain: 3000.0×1

## Convergence Analysis

In [ ]:
# Leapfrog Convergence Study for Medium Grid
# Test if error decreases as O(dt^2) until hitting machine precision

# Select medium model
model = models['medium']
p = model['p']
Nx = p['Nx']
Nz = p['Nz']
N = Nx * Nz

# Define input function
eval_u = lambda t: (p['dx'] * p['dz']) * eval_u_Sonar_20(t)

# Timestep factors to test (from coarse to very fine)
dt_factors = [0.8, 0.4, 0.2, 0.1, 0.05, 0.025, 0.0125, 0.00625, 0.003125, 0.0015625]
print("="*80)
print("LEAPFROG CONVERGENCE STUDY - MEDIUM GRID")
print("="*80)
print(f"Testing {len(dt_factors)} timestep sizes from {dt_factors[0]:.4f}× to {dt_factors[-1]:.6f}× CFL")
print(f"CFL limit: {model['max_dt_FE']*1e6:.3f} μs")
print(f"Runtime: {model['t_stop']:.2f} s")
print("="*80)

# Storage for results
results = []

for i, dt_factor in enumerate(dt_factors):
    # Force garbage collection before each run to ensure clean memory state
    gc.collect()
    
    dt = model['max_dt_FE'] * dt_factor
    num_steps = int(np.ceil((model['t_stop'] - model['t_start']) / dt))
    
    print(f"\n[{i+1}/{len(dt_factors)}] Running dt = {dt_factor:.6f}× CFL ({dt*1e6:.4f} μs)")
    print(f"  Steps: {num_steps:,}")
    
    # Run Leapfrog
    t0 = time.perf_counter()
    X, t = LeapfrogSolver(
        eval_f=eval_f_Sonar,
        x_start=model['x_start'],
        p=p,
        eval_u=eval_u,
        NumIter=num_steps,
        dt=dt,
        visualize=False,
        verbose=True
    )
    runtime = time.perf_counter() - t0
    
    # Extract hydrophone 2 signal at final time
    y_h2_final = eval_g_Sonar(X[:, -1].reshape(-1, 1), p)[1, 0]
    t_final = t[-1]
    
    # Free memory IMMEDIATELY - delete X and t before storing results
    del X, t
    gc.collect()
    
    # Store results (WITHOUT the large X and t arrays)
    results.append({
        'dt_factor': dt_factor,
        'dt': dt,
        'num_steps': num_steps,
        'runtime': runtime,
        'y_h2_final': y_h2_final,
        't_final': t_final
    })
    
    print(f"  Runtime: {runtime:.2f} s")
    print(f"  Final time: {t_final:.6f} s")
    print(f"  H2(t_end): {y_h2_final:.10e}")

print("\n" + "="*80)
print("Computing convergence metrics...")
print("="*80)

# Use finest resolution as "truth"
y_ref = results[-1]['y_h2_final']
print(f"\nReference (finest dt = {dt_factors[-1]:.6f}×): H2 = {y_ref:.10e}")

# Calculate machine precision threshold (Option 2: Relative to reference value)
machine_eps = np.finfo(float).eps  # 2.22e-16
safety_factor = 1e3  # 1000× epsilon for conservative estimate
machine_threshold = max(1e-13, abs(y_ref) * machine_eps * safety_factor)

print(f"\nMachine precision threshold: {machine_threshold:.3e}")
print(f"  (≈ {safety_factor:.0f}× machine epsilon = {machine_eps:.2e})")
print(f"  Errors below this are numerical noise")

# Compute errors and convergence rates
for i, r in enumerate(results):
    r['abs_error'] = abs(r['y_h2_final'] - y_ref)
    r['rel_error'] = abs(r['abs_error'] / y_ref) if abs(y_ref) > 1e-15 else r['abs_error']
    
    # Estimate convergence order from previous result
    # Only compute if BOTH errors are above machine precision threshold
    if i > 0:
        dt_ratio = results[i-1]['dt'] / r['dt']
        prev_error = results[i-1]['abs_error']
        curr_error = r['abs_error']
        
        # Check if we're in the meaningful regime (above machine precision)
        if (prev_error > machine_threshold and 
            curr_error > machine_threshold and 
            curr_error < prev_error):  # Error actually decreasing
            error_ratio = prev_error / curr_error
            r['conv_order'] = np.log(error_ratio) / np.log(dt_ratio)
        else:
            r['conv_order'] = np.nan  # Mark as invalid
    else:
        r['conv_order'] = np.nan
    
    print(f"\ndt = {r['dt_factor']:.6f}×: H2 = {r['y_h2_final']:.10e}")
    print(f"  Abs Error: {r['abs_error']:.3e}")
    print(f"  Rel Error: {r['rel_error']:.3e}")
    

    if not np.isnan(r['conv_order']):    print("\n✓ No plateau detected - still converging at finest resolution")

        indicator = '✓' if 1.8 <= r['conv_order'] <= 2.2 else '⚠️'else:

        print(f"  Conv Order: {r['conv_order']:.2f} {indicator}")    print(f"    Finer timesteps will not improve accuracy")

    else:    print(f"    Error level: {results[plateau_idx]['abs_error']:.3e}")

        if i > 0 and r['abs_error'] <= machine_threshold:    print(f"\n⚠️  Machine precision reached at dt = {results[plateau_idx]['dt_factor']:.6f}×")

            print(f"  Conv Order: N/A (machine precision regime)")if plateau_idx is not None:

        else:

            print(f"  Conv Order: N/A")        break

        plateau_idx = i

# Detect where we hit machine precision limit    if results[i]['abs_error'] <= machine_threshold:

plateau_idx = Nonefor i in range(len(results)-1):

LEAPFROG CONVERGENCE STUDY - MEDIUM GRID
Testing 10 timestep sizes from 0.8000× to 0.001563× CFL
CFL limit: 3535.534 μs
Runtime: 1.50 s

[1/10] Running dt = 0.800000× CFL (2828.4271 μs)
  Steps: 531
Running Leapfrog solver (stable for wave equations)...
Bootstrapping with RK4 for first step...
Running 530 leapfrog steps...
  Progress: 10.0%
  Progress: 20.0%
  Progress: 29.9%
  Progress: 39.9%
  Progress: 49.9%
  Progress: 59.9%
  Progress: 69.9%
  Progress: 79.8%
  Progress: 89.8%
  Progress: 39.9%
  Progress: 49.9%
  Progress: 59.9%
  Progress: 69.9%
  Progress: 79.8%
  Progress: 89.8%
  Progress: 99.8%
Leapfrog integration complete!
  Runtime: 0.41 s
  Final time: 1.501895 s
  H2(t_end): -1.1546793712e-05

[2/10] Running dt = 0.400000× CFL (1414.2136 μs)
  Steps: 1,061
Running Leapfrog solver (stable for wave equations)...
Bootstrapping with RK4 for first step...
Running 1060 leapfrog steps...
  Progress: 10.0%
  Progress: 99.8%
Leapfrog integration complete!
  Runtime: 0.41 s
  Fin

C:\Users\Manny\AppData\Local\Temp\ipykernel_88020\3333734389.py:89: RuntimeWarning: divide by zero encountered in scalar divide
  error_ratio = results[i-1]['abs_error'] / r['abs_error']


In [ ]:
# Visualize convergence behavior
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Extract data for plotting (exclude finest as it's the reference)
dts = [r['dt']*1e6 for r in results[:-1]]  # Convert to μs
abs_errors = [r['abs_error'] for r in results[:-1]]
rel_errors = [r['rel_error'] for r in results[:-1]]
conv_orders = [r['conv_order'] for r in results[1:-1]]  # Skip first (no order)
runtimes = [r['runtime'] for r in results[:-1]]

# --- Plot 1: Absolute Error vs dt (log-log) ---
ax1.loglog(dts, abs_errors, 'o-', linewidth=2, markersize=8, label='Actual Error', color='#2E86AB')

# Plot reference O(dt^2) line
dt_theory = np.array([dts[0], dts[-1]])
error_theory = abs_errors[0] * (dt_theory / dts[0])**2
ax1.loglog(dt_theory, error_theory, '--', linewidth=2, label='O(dt²) reference', color='gray', alpha=0.7)

ax1.set_xlabel('Timestep dt (μs)', fontsize=12)
ax1.set_ylabel('Absolute Error |H2(t_end) - H2_ref|', fontsize=12)
ax1.set_title('Convergence: Error vs Timestep (log-log)', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3, which='both')
ax1.invert_xaxis()  # Smaller dt on right

# Add annotation for machine precision
if plateau_idx is not None:
    ax1.axhline(y=results[plateau_idx]['abs_error'], color='red', 
                linestyle=':', linewidth=2, alpha=0.5, label='Machine Precision Limit')
    ax1.text(dts[0], results[plateau_idx]['abs_error']*1.5, 
             'Machine Precision Limit', fontsize=10, color='red')

# --- Plot 2: Convergence Order vs dt ---
dt_mid = [r['dt']*1e6 for r in results[1:-1]]
conv_orders = [r['conv_order'] for r in results[1:-1]]

# Filter out NaN values (machine precision regime)
valid_indices = [i for i, co in enumerate(conv_orders) if not np.isnan(co)]
dt_mid_valid = [dt_mid[i] for i in valid_indices]
conv_orders_valid = [conv_orders[i] for i in valid_indices]

if len(conv_orders_valid) > 0:
    ax2.semilogx(dt_mid_valid, conv_orders_valid, 'o-', linewidth=2, markersize=8, 
                 color='#F18F01', label='Computed Order')
    ax2.axhline(y=2.0, color='gray', linestyle='--', linewidth=2, alpha=0.7, 
                label='Expected: 2nd order')
    ax2.set_xlabel('Timestep dt (μs)', fontsize=12)
    ax2.set_ylabel('Estimated Convergence Order', fontsize=12)
    ax2.set_title('Convergence Order (should approach 2 for Leapfrog)', fontsize=13, fontweight='bold')
    ax2.legend(fontsize=11, loc='best')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim([0, 3.5])
    ax2.invert_xaxis()
    
    # Add annotation if some points were excluded
    num_excluded = len(conv_orders) - len(conv_orders_valid)
    if num_excluded > 0:
        ax2.text(0.05, 0.95, f'{num_excluded} point(s) excluded\n(machine precision regime)', 
                transform=ax2.transAxes, fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))
else:
    ax2.text(0.5, 0.5, 'All timesteps at machine precision\n(no convergence order measurable)', 
            transform=ax2.transAxes, fontsize=11, ha='center', va='center',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.6))
    ax2.set_xlabel('Timestep dt (μs)', fontsize=12)
    ax2.set_ylabel('Estimated Convergence Order', fontsize=12)
    ax2.set_title('Convergence Order (machine precision regime)', fontsize=13, fontweight='bold')
    ax2.grid(True, alpha=0.3)

# --- Plot 3: Runtime vs Accuracy ---
ax3.loglog(abs_errors, runtimes, 'o-', linewidth=2, markersize=8, color='#A23B72')
ax3.set_xlabel('Absolute Error', fontsize=12)
ax3.set_ylabel('Runtime (s)', fontsize=12)
ax3.set_title('Computational Cost vs Accuracy', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3, which='both')

# Add annotations for key points
for i, (err, rt, dt_f) in enumerate(zip(abs_errors[::2], runtimes[::2], [results[j]['dt_factor'] for j in range(0, len(results)-1, 2)])):
    ax3.annotate(f'{dt_f:.3f}×', xy=(err, rt), xytext=(5, 5), 
                textcoords='offset points', fontsize=9, alpha=0.7)

# --- Plot 4: H2 value vs dt (convergence to final value) ---
h2_values = [r['y_h2_final'] for r in results]
dt_all = [r['dt']*1e6 for r in results]
ax4.semilogx(dt_all, h2_values, 'o-', linewidth=2, markersize=8, color='#2E86AB')
ax4.axhline(y=y_ref, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Reference Value')
ax4.set_xlabel('Timestep dt (μs)', fontsize=12)
ax4.set_ylabel('H2(t_end) Pressure Value', fontsize=12)
ax4.set_title('H2 Final Value Convergence', fontsize=13, fontweight='bold')
ax4.legend(fontsize=11)
ax4.grid(True, alpha=0.3)
ax4.invert_xaxis()

# Add shaded region showing machine precision
if abs(y_ref) > 0:
    eps_machine = np.finfo(float).eps * abs(y_ref) * 10  # ~10× machine epsilon
    ax4.axhspan(y_ref - eps_machine, y_ref + eps_machine, alpha=0.1, color='red')
    ax4.text(dt_all[0], y_ref, f'  Machine ε ≈ {eps_machine:.2e}', 
             fontsize=9, color='red', verticalalignment='center')

plt.tight_layout()
plt.savefig('leapfrog_convergence_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "="*80)
print("CONVERGENCE SUMMARY")
print("="*80)
print(f"Finest resolution: dt = {results[-1]['dt_factor']:.6f}× CFL ({results[-1]['dt']*1e6:.4f} μs)")
print(f"Reference value: H2(t_end) = {y_ref:.10e}")
print(f"\nConvergence quality:")

# Compute average convergence order in pre-plateau region
if plateau_idx is not None:
    valid_orders = [results[i]['conv_order'] for i in range(1, plateau_idx) if not np.isnan(results[i]['conv_order'])]
else:
    valid_orders = [r['conv_order'] for r in results[1:-1] if not np.isnan(r['conv_order'])]

if valid_orders:
    avg_order = np.mean(valid_orders)
    print(f"\nAverage convergence order: {avg_order:.2f}")
    
    if 1.8 <= avg_order <= 2.2:
        print(f"  ✓ Excellent! Close to theoretical O(dt²) for Leapfrog")
    elif 1.5 <= avg_order <= 2.5:
        print(f"  ✓ Good - within expected range for 2nd order method")
    else:
        print(f"  ⚠️  Unexpected order - should be near 2.0 for Leapfrog")

if plateau_idx is not None:
    print(f"\n⚠️  Machine precision reached at dt = {results[plateau_idx]['dt_factor']:.6f}×")
    print(f"    Error level: {results[plateau_idx]['abs_error']:.3e}")
    print(f"    Finer timesteps will not improve accuracy")
    print(f"  • Converges until dt ≈ {results[plateau_idx]['dt_factor']:.6f}× CFL")
    print(f"  • Machine precision limit: ~{results[plateau_idx]['abs_error']:.2e}")
    print(f"  • Meaningful range: dt > {results[plateau_idx]['dt']*1e6:.3f} μs")
else:
    print(f"\n✓ No plateau detected - still converging at finest resolution")
    print(f"  • Still converging at finest resolution tested")
    print(f"  • May need even smaller dt to reach machine precision")

print("="*80)

IndentationError: unexpected indent (2191471687.py, line 114)